In [1]:
with open("./data/example10.txt", "r") as f:
    example = f.read()

In [96]:
with open('./data/day10.txt', 'r') as f:
    data = f.read()

In [3]:
import re

def parse_input(input_str):
    lights = []
    switches = []
    joltage = []
    for line in input_str.splitlines():
        sections = line.split(' ')
        light = list(sections[0].strip('[').strip(']'))
        switch = []
        for section in sections[1:-1]:
            numbers = re.findall(r"\(([\d,]+)\)", section)
            if numbers:
                num_list = tuple(int(n) for n in numbers[0].split(","))
            switch.append(num_list)
        row_joltage = sections[-1]
        lights.append(light)
        switches.append(switch)
        joltages = re.findall(r"(\d+)", row_joltage)
        row_joltage = tuple(int(j) for j in joltages)
        joltage.append(row_joltage)
    return lights, switches, joltage

In [4]:
from functools import lru_cache
import heapq

def update_lights(light_state, switch):
    lights = list(light_state)
    for lamp in switch:
        if lights[lamp] == ".":
            lights[lamp] = "#"
        else:
            lights[lamp] = "."
    return tuple(lights)

def min_steps(switches_goal):
    switch_options, goal = switches_goal
    current_lamp = tuple("." for _ in goal)
    state = [(0, current_lamp, -1)]
    visited = set()
    
    while state:
        steps, lamp_state, last_switch = heapq.heappop(state)
        
        # Skip if we've already visited this state
        if lamp_state in visited:
            continue
        visited.add(lamp_state)
        
        if lamp_state == tuple(goal):
            return steps
        for i, switch in enumerate(switch_options):
            if i == last_switch:
                continue
            new_lamp_state = update_lights(lamp_state, switch)
            
            # Skip if this would create a loop (already visited state)
            if new_lamp_state in visited:
                continue
            
            new_steps = steps + 1
            heapq.heappush(state, (new_steps, new_lamp_state, i))


In [5]:
sub_stepcounter = []
paths = []
lights, switch, joltage = parse_input(example)

for lights_row, switch_row in zip(lights, switch):
    switch_goal = (switch_row, lights_row)
    step= min_steps(switch_goal)
    sub_stepcounter.append(step)
sum(sub_stepcounter)

7

In [163]:
sub_stepcounter = []
paths = []
lights, switch, joltage = parse_input(data)

for lights_row, switch_row in zip(lights, switch):
    switch_goal = (switch_row, lights_row)
    step = min_steps(switch_goal)
    sub_stepcounter.append(step)
sum(sub_stepcounter)

532

In [6]:
import dask
from dask.distributed import Client, progress

client = Client(threads_per_worker=1, n_workers=4)

In [7]:
sub_stepcounter = []
paths = []
lights, switch, joltage = parse_input(data)

for lights_row, switch_row in zip(lights, switch):
    switch_goal = (switch_row, lights_row)
    lazy_result = dask.delayed(min_steps)(switch_goal)
    sub_stepcounter.append(lazy_result)

final_stepcount = dask.delayed(sum)(sub_stepcounter)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 61.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55184,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 61.52 GiB
Comm: tcp://127.0.0.1:55203,Total threads: 1
Dashboard: http://127.0.0.1:55204/status,Memory: 15.38 GiB
Nanny: tcp://127.0.0.1:55187,


In [8]:
result = client.compute(final_stepcount)

In [9]:
client.gather(result)

532

In [10]:
client.close()

In [160]:
def min_steps_with_joltage(switches, goal):
    """"Let's try to create someting with linear algebra / linear programming"""
    import numpy as np
    from scipy.optimize import linprog
    switch_array = np.zeros((len(goal), len(switches)))
    for i, switch in enumerate(switches):
        for lamp in switch:
            switch_array[lamp][i] = 1
    B= np.array(goal)

    result = linprog(
        A_eq=switch_array,
        b_eq=B,
        c=np.ones(len(switches)),
        bounds =(0, None),
        integrality=1,
        method="highs",
    )
    return round(sum(result.x))

In [161]:
sub_stepcounter = []
lights, switch, joltage = parse_input(example)

for  switch_row, joltage_row in zip( switch, joltage):
    step = min_steps_with_joltage(switch_row, joltage_row)
    sub_stepcounter.append(step)
sum(sub_stepcounter)

33

In [162]:
sub_stepcounter = []
lights, switch, joltage = parse_input(data)

for switch_row, joltage_row in zip(switch, joltage):
    step = min_steps_with_joltage(switch_row, joltage_row)
    sub_stepcounter.append(step)
sum(sub_stepcounter)

18387